decoder transformer层的linear2层（dtl2）替换为近似矩阵乘法, 32\*32\*128, 128\*64, 32\*32\*64

In [7]:
import numpy as np
import os
import matmul as mm
import math_util as mu
# import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [8]:
# root_path = '/data/hdr/csi_transformer/pic/'
# print(root_path)
# root_path2 = '/'.join(root_path.split("/")[:-2])
# # sys.path.append(root_path)
# print(root_path2)

In [9]:
# method = METHOD_MITHRAL
# method = METHOD_PQ
method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE

In [10]:
linear_name = 'dtl2'
feedback_bits = 256
ncodebooks = 16  # max:128
ncentroids = 256
if method == METHOD_MITHRAL:
    ncentroids = 16
train_sam_num = 3000 # 训练集样本数
test_sam_num = 1000 # 测试集样本数
batch_size = 32
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0

In [11]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= ''
    featurepath_train= ''
    data_to_fcpath_test = ''
    featurepath_test = ''
elif host_name == 'jm-System-Product-Name':
    dir_train = os.path.join('/data/hdr/transformer_data/joined', 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join('/data/hdr/transformer_data/joined', 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'dx_linear2in_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'dx_linear2_y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'dx_linear2out_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'dx_linear2in_test_f%i_sam%i.npy' % (feedback_bits, test_sam_num)
    featurepath_test = 'dx_linear2out_test_f%i_sam%i.npy' % (feedback_bits, test_sam_num)
    # y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dirs")


weightpath = 'dx_linear2_w_f256.npy'
biaspath = 'dx_linear2_b_f256.npy'
dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, linear_name)
try:
    os.mkdir(dir_result)
except FileExistsError:
    pass 


In [12]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])


running method:  Exact


In [13]:
# type(est3)

In [14]:
x_test = np.load(dir_test+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [15]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1]) #AMM字典模式需要复原y大小
print("y_out_last_re.shape: ", y_out_last_re.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
if method == METHOD_EXACT:
    train_sam_num = 0 # 训练集样本数
np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, ncodebooks, ncentroids)), y_out_last_re)
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[ 2.3810785e-02 -1.3505399e-02  1.0349123e-02 ... -5.5558234e-02
  -1.1642658e-01  1.7807508e-01]
 [ 4.0933378e-03 -1.9378886e-02  1.8495575e-01 ... -4.3654874e-01
  -9.6737497e-02  3.1615591e-01]
 [ 8.0945021e-01  4.5894659e-01  5.9334582e-01 ...  7.7175623e-01
  -1.3781214e+00  5.6198630e+00]
 ...
 [-1.2953599e-02  2.1025605e-01  1.4551432e-01 ... -8.5038073e-02
  -1.6469622e-01  9.5378011e-01]
 [ 1.2768504e-01  2.0935626e-01  1.4957690e-01 ... -9.1138653e-02
   1.4657140e-01 -1.8523394e-01]
 [ 9.9036418e-02  2.1650170e-01  1.3826513e-01 ... -7.3654629e-02
  -3.0525941e-01  9.0723741e-01]]
y_out_last.shape:  (1024000, 64)
y_out_last_re.shape:  (1000, 32, 32, 64)
